# IMPORT

In [46]:
import pandas as pd
from _data_manager import DataManager
from _trade import Trade
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler




<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# DATA ANALYSIS

To view the data, launch the dashboard, which offers numerous data visualizations.

In [34]:
file_path = 'EGB_historical_yield.csv'
data_manager = DataManager(file_path)


            Germany_5y-Germany_10y  Germany_5y-Germany_30y  \
date                                                         
2016-01-15                   -70.3                  -147.6   
2016-01-18                   -70.2                  -147.8   
2016-01-19                   -71.5                  -148.9   
2016-01-20                   -67.9                  -142.3   
2016-01-21                   -68.9                  -145.4   
...                            ...                     ...   
2024-02-09                    -4.6                   -23.2   
2024-02-12                    -5.7                   -25.3   
2024-02-13                    -3.6                   -20.0   
2024-02-14                    -3.5                   -20.7   
2024-02-15                    -2.8                   -19.9   

            Germany_5y-Belgium_5y  Germany_5y-Belgium_10y  \
date                                                        
2016-01-15                  -14.3                  -100.2   
2016-01-18

/Users/martialgil/Desktop/COURS/Backtesting/Govies_Quant_Trading/Project/data_manager.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_series = (df[colA] - df[colB]).rename(column_name)
/Users/martialgil/Desktop/COURS/Backtesting/Govies_Quant_Trading/Project/data_manager.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  diff_series = (df[colA] - df[colB]).rename(column_name)
/Users/martialgil/Desktop/COURS/Backtesting/Govies_Quant_Trading/Project/data_manager.py:84: PerformanceWarning: DataFrame is highly fragmented.

In [4]:
data_manager.data.shape

(2110, 21)

In [5]:
data_manager.data.head()

,Germany_5y,Germany_10y,Germany_30y,Belgium_5y,Belgium_10y,Belgium_30y,Austria_5y,Austria_10y,Austria_30y,Netherlands_5y,...,Netherlands_30y,France_5y,France_10y,France_30y,Spain_5y,Spain_10y,Spain_30y,Italy_5y,Italy_10y,Italy_30y
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-16.4,53.9,131.2,-2.1,83.8,189.7,-8,77.7,160.4,-10.4,...,143.6,-10.4,66.1,143.6,65.4,175.0,291.2,56.1,156.4,264.9
2016-01-18,-16.6,53.6,131.2,-3.3,83.5,190.0,-8,77.4,160.7,-10.6,...,143.9,-10.6,66.0,143.9,64.0,174.0,291.5,56.6,156.4,266.9
2016-01-19,-16.7,54.8,132.2,-4.1,84.0,189.9,-8,78.3,161.8,-11.3,...,145.1,-11.3,66.1,145.1,60.5,170.1,288.2,55.3,155.7,265.2
2016-01-20,-19.8,48.1,122.5,-6.0,96.3,184.2,-11,73.2,153.1,-13.8,...,135.9,-13.8,60.9,135.9,65.7,178.2,296.7,62.2,164.9,275.9
2016-01-21,-24.0,44.9,121.4,-9.8,91.5,180.5,-16,69.6,153.0,-18.2,...,134.8,-18.2,57.0,134.8,60.2,171.9,291.5,54.4,156.1,269.5


In [6]:
data_manager.data.dtypes

Germany_5y         float64
Germany_10y        float64
Germany_30y        float64
Belgium_5y         float64
Belgium_10y        float64
Belgium_30y        float64
Austria_5y           int64
Austria_10y        float64
Austria_30y        float64
Netherlands_5y     float64
Netherlands_10y    float64
Netherlands_30y    float64
France_5y          float64
France_10y         float64
France_30y         float64
Spain_5y           float64
Spain_10y          float64
Spain_30y          float64
Italy_5y           float64
Italy_10y          float64
Italy_30y          float64
dtype: object

For the Austria 5-year rates, we only have `int64` data types, which likely indicates a lack of precision in this measurement compared to others. Given 2110 rows, the probability of having only integer numbers is virtually zero.

In [7]:
data_manager.data.isnull().sum().sort_values().head(3)

Germany_5y    0
Italy_5y      0
Spain_30y     0
dtype: int64

There are no missing data, which is good news.

In [15]:
import pandas as pd


def create_all_dict(dataset):
    resultat = {}
    for position in dataset.index:
        resultat[position] = comparer_criteres(position,dataset)
    return resultat


def comparer_criteres(ligne, dataset):
    """
    Retourne un dictionnaire où chaque clé est une colonne du dataset, et la valeur est un dictionnaire avec deux clés :
    "mieux_classe" et "moins_classe", contenant les listes des colonnes mieux classées et moins bien classées respectivement.
    
    :param ligne: L'index de la ligne à analyser dans le dataset.
    :param dataset: Le DataFrame Pandas contenant le dataset.
    :return: Dictionnaire de comparaison des critères.
    """
    
    resultat = {}
    valeurs_ligne = dataset.loc[ligne]
    
    for colonne in dataset.columns:
        print(colonne)
        mieux_classe = []
        moins_classe = []
        
        for autre_colonne in dataset.columns:
            if autre_colonne == colonne:
                continue  # On ne compare pas la colonne avec elle-même.
            if valeurs_ligne[colonne] > valeurs_ligne[autre_colonne]:
                mieux_classe.append(autre_colonne)
            elif valeurs_ligne[colonne] < valeurs_ligne[autre_colonne]:
                moins_classe.append(autre_colonne)
            else:
                continue #si la colonne est classé comme la notre
        
        resultat[colonne] = {
            "mieux_classe": mieux_classe,
            "moins_classe": moins_classe
        }
    
    return resultat

# Exemple d'utilisation
# Supposons que `df` est votre DataFrame Pandas contenant le dataset.
# ligne_index est l'index de la ligne que vous voulez analyser.
# resultat = comparer_criteres(ligne_index, df)
# print(resultat)


In [17]:
dic =create_all_dict( data_manager.rank_data)


Germany_5y
Germany_10y
Germany_30y
Belgium_5y
Belgium_10y
Belgium_30y
Austria_5y
Austria_10y
Austria_30y
Netherlands_5y
Netherlands_10y
Netherlands_30y
France_5y
France_10y
France_30y
Spain_5y
Spain_10y
Spain_30y
Italy_5y
Italy_10y
Italy_30y
Germany_5y
Germany_10y
Germany_30y
Belgium_5y
Belgium_10y
Belgium_30y
Austria_5y
Austria_10y
Austria_30y
Netherlands_5y
Netherlands_10y
Netherlands_30y
France_5y
France_10y
France_30y
Spain_5y
Spain_10y
Spain_30y
Italy_5y
Italy_10y
Italy_30y
Germany_5y
Germany_10y
Germany_30y
Belgium_5y
Belgium_10y
Belgium_30y
Austria_5y
Austria_10y
Austria_30y
Netherlands_5y
Netherlands_10y
Netherlands_30y
France_5y
France_10y
France_30y
Spain_5y
Spain_10y
Spain_30y
Italy_5y
Italy_10y
Italy_30y
Germany_5y
Germany_10y
Germany_30y
Belgium_5y
Belgium_10y
Belgium_30y
Austria_5y
Austria_10y
Austria_30y
Netherlands_5y
Netherlands_10y
Netherlands_30y
France_5y
France_10y
France_30y
Spain_5y
Spain_10y
Spain_30y
Italy_5y
Italy_10y
Italy_30y
Germany_5y
Germany_10y
Germany_3

In [23]:
dic[data_manager.rank_data.index[0]]

{'Germany_5y': {'mieux_classe': [],
  'moins_classe': ['Germany_10y',
   'Germany_30y',
   'Belgium_5y',
   'Belgium_10y',
   'Belgium_30y',
   'Austria_5y',
   'Austria_10y',
   'Austria_30y',
   'Netherlands_5y',
   'Netherlands_10y',
   'Netherlands_30y',
   'France_5y',
   'France_10y',
   'France_30y',
   'Spain_5y',
   'Spain_10y',
   'Spain_30y',
   'Italy_5y',
   'Italy_10y',
   'Italy_30y']},
 'Germany_10y': {'mieux_classe': ['Germany_5y',
   'Belgium_5y',
   'Austria_5y',
   'Netherlands_5y',
   'France_5y'],
  'moins_classe': ['Germany_30y',
   'Belgium_10y',
   'Belgium_30y',
   'Austria_10y',
   'Austria_30y',
   'Netherlands_10y',
   'Netherlands_30y',
   'France_10y',
   'France_30y',
   'Spain_5y',
   'Spain_10y',
   'Spain_30y',
   'Italy_5y',
   'Italy_10y',
   'Italy_30y']},
 'Germany_30y': {'mieux_classe': ['Germany_5y',
   'Germany_10y',
   'Belgium_5y',
   'Belgium_10y',
   'Austria_5y',
   'Austria_10y',
   'Netherlands_5y',
   'Netherlands_10y',
   'France_5y',


In [22]:
data_manager.rank_data.index[0]

Timestamp('2016-01-15 00:00:00')

In [24]:
import itertools

# Liste initiale
liste_initiale = [1, 2, 3]

# Utiliser itertools.combinations pour obtenir toutes les paires possibles
paires_possibles = list(itertools.combinations(liste_initiale, 2))

paires_possibles


[(1, 2), (1, 3), (2, 3)]

In [26]:
import pandas as pd
import itertools

# Créer un DataFrame de pandas pour l'exemple
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}
df = pd.DataFrame(data)

# Obtenir les noms des colonnes
noms_colonnes = df.columns

# Utiliser itertools.combinations pour obtenir toutes les paires possibles de noms de colonnes
paires_colonnes = list(itertools.combinations(noms_colonnes, 2))

paires_colonnes


# Créer un nouveau DataFrame pour stocker les résultats
df_diff = pd.DataFrame()

# Calculer la différence pour chaque paire de colonnes et ajouter au nouveau DataFrame
for colA, colB in paires_colonnes:
    nom_colonne = f"{colA}-{colB}"
    df_diff[nom_colonne] = df[colA] - df[colB]

df_diff


,A-B,A-C,B-C
0,-3,-6,-3
1,-3,-6,-3
2,-3,-6,-3


In [35]:
data_manager.spread_yield

,Germany_5y-Germany_10y,Germany_5y-Germany_30y,Germany_5y-Belgium_5y,Germany_5y-Belgium_10y,Germany_5y-Belgium_30y,Germany_5y-Austria_5y,Germany_5y-Austria_10y,Germany_5y-Austria_30y,Germany_5y-Netherlands_5y,Germany_5y-Netherlands_10y,...,Spain_10y-Spain_30y,Spain_10y-Italy_5y,Spain_10y-Italy_10y,Spain_10y-Italy_30y,Spain_30y-Italy_5y,Spain_30y-Italy_10y,Spain_30y-Italy_30y,Italy_5y-Italy_10y,Italy_5y-Italy_30y,Italy_10y-Italy_30y
date,,,,,,,,,,,,,,,,,,,,,
2016-01-15,-70.3,-147.6,-14.3,-100.2,-206.1,-8.4,-94.1,-176.8,-6.0,-82.5,...,-116.2,118.9,18.6,-89.9,235.1,134.8,26.3,-100.3,-208.8,-108.5
2016-01-18,-70.2,-147.8,-13.3,-100.1,-206.6,-8.6,-94.0,-177.3,-6.0,-82.6,...,-117.5,117.4,17.6,-92.9,234.9,135.1,24.6,-99.8,-210.3,-110.5
2016-01-19,-71.5,-148.9,-12.6,-100.7,-206.6,-8.7,-95.0,-178.5,-5.4,-82.8,...,-118.1,114.8,14.4,-95.1,232.9,132.5,23.0,-100.4,-209.9,-109.5
2016-01-20,-67.9,-142.3,-13.8,-116.1,-204.0,-8.8,-93.0,-172.9,-6.0,-80.7,...,-118.5,116.0,13.3,-97.7,234.5,131.8,20.8,-102.7,-213.7,-111.0
2016-01-21,-68.9,-145.4,-14.2,-115.5,-204.5,-8.0,-93.6,-177.0,-5.8,-81.0,...,-119.6,117.5,15.8,-97.6,237.1,135.4,22.0,-101.7,-215.1,-113.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-09,-4.6,-23.2,-32.1,-66.9,-116.2,-35.6,-59.2,-77.9,-20.3,-38.5,...,-60.0,-5.1,-59.7,-118.2,54.9,0.3,-58.2,-54.6,-113.1,-58.5
2024-02-12,-5.7,-25.3,-32.0,-66.8,-115.9,-35.7,-59.7,-79.3,-20.7,-39.5,...,-60.8,-3.1,-58.6,-117.6,57.7,2.2,-56.8,-55.5,-114.5,-59.0
2024-02-13,-3.6,-20.0,-32.0,-63.7,-109.0,-36.4,-57.1,-72.7,-20.7,-37.7,...,-57.1,-6.4,-60.1,-115.8,50.7,-3.0,-58.7,-53.7,-109.4,-55.7


In [45]:
data_manager.data["2016-01-15":"2016-01-21"].loc[:,['Germany_30y']]

,Germany_30y
date,
2016-01-15,131.2
2016-01-18,131.2
2016-01-19,132.2
2016-01-20,122.5
2016-01-21,121.4


In [47]:
trade = Trade("Germany_30y",1,1,"2016-01-15","2016-01-21")

In [48]:
trade.daily_Pand_L(data_manager.data)

ValueError: Cannot set a DataFrame with multiple columns to the single column PandL_Germany_30y

### Données économiques que l'on pourrait rajouter pour aller plus loin :

- Inflation
- Chômage
- PIB (Produit Intérieur Brut)
- Indice des prix à la consommation
- Indice des prix à la production (IPP)
- Balance commerciale
- Indices boursiers
- Indicateurs de sentiment de marché


In [36]:
def subsets(lst):
    if len(lst) == 0:
        return [[]]  # Le sous-ensemble vide est toujours présent

    first_elem = lst[0]
    rest_subsets = subsets(lst[1:])  # Appel récursif pour obtenir les sous-ensembles du reste de la liste
    new_subsets = []
    for subset in rest_subsets:
        new_subsets.append(subset)  # Ajouter le sous-ensemble sans le premier élément
        new_subsets.append([first_elem] + subset)  # Ajouter le sous-ensemble avec le premier élément
    return new_subsets

# Exemple d'utilisation
liste = [1, 2, 3]
print("Tous les sous-ensembles de", liste, "sont:", subsets(liste))


Tous les sous-ensembles de [1, 2, 3] sont: [[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]


In [40]:
liste+[1]

[1, 2, 3, 1]

In [38]:
liste[0]

1